<a href="https://colab.research.google.com/github/bthu207/group-project/blob/main/hex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# === [1] Import packages ===
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import json

In [3]:
# === [2] Đọc file CSV thực ===
csv_paths = [
    '/content/drive/MyDrive/[DS-B3] group project/processed_folktales.csv',
    '/content/drive/MyDrive/[DS-B3] group project/processed_articles_sep2025.csv',
    '/content/drive/MyDrive/[DS-B3] group project/processed_articles_aug_oct_2025.csv'
    ]
# Đọc và gộp nhiều file lại 1 DataFrame duy nhất
df_list = []
for path in csv_paths:
    try:
        temp_df = pd.read_csv(path)
        print(f"✅ Loaded file: {path}")
        print(f"   Cột: {list(temp_df.columns)}")
        df_list.append(temp_df)
    except Exception as e:
        print(f" Lỗi khi đọc {path}: {e}")

# Gộp tất cả thành 1 DataFrame duy nhất
df = pd.concat(df_list, ignore_index=True)
print("\nTổng số bài báo sau khi gộp:", len(df))
print("Danh sách cột hợp nhất:", list(df.columns))


✅ Loaded file: /content/drive/MyDrive/[DS-B3] group project/processed_folktales.csv
   Cột: ['title', 'content', 'url', 'section', 'source']
✅ Loaded file: /content/drive/MyDrive/[DS-B3] group project/processed_articles_sep2025.csv
   Cột: ['url', 'title', 'content', 'published_time', 'source', 'title_clean', 'content_clean', 'label']
✅ Loaded file: /content/drive/MyDrive/[DS-B3] group project/processed_articles_aug_oct_2025.csv
   Cột: ['title', 'content', 'url', 'section', 'source', 'published_time']

Tổng số bài báo sau khi gộp: 2791
Danh sách cột hợp nhất: ['title', 'content', 'url', 'section', 'source', 'published_time', 'title_clean', 'content_clean', 'label']


In [4]:
import pandas as pd

def load_articles(csv_path):
    """
    Đọc file CSV bất kỳ và chuẩn hóa tên cột về cấu trúc chung.
    """
    df = pd.read_csv(csv_path)

    # Tạo dict mapping: tên cột có thể gặp -> cột chuẩn
    name_map = {
        "url": ["url", "link", "article_url"],
        "title": ["title", "headline", "title_clean"],
        "content": ["content", "body", "article_content", "content_clean"],
        "published_time": ["published_time", "date", "datetime", "created_at"],
        "category": ["category", "section", "topic"],
        "label": ["label", "target", "age_group", "safe_label"],
        "section": ["section"]
    }

    # Hàm tìm cột thật từ tên gần giống
    def find_column(possibles, columns):
        for p in possibles:
            for c in columns:
                if p.lower() == c.lower():
                    return c
        return None

    selected = {}
    for std_col, poss in name_map.items():
        real_col = find_column(poss, df.columns)
        if real_col:
            selected[std_col] = real_col

    df_clean = pd.DataFrame()
    for std_col, real_col in selected.items():
        df_clean[std_col] = df[real_col]

    # Thêm cột còn thiếu (giá trị rỗng)
    for col in name_map.keys():
        if col not in df_clean.columns:
            df_clean[col] = None

    print(f"Loaded {csv_path} — mapping: {selected}")
    print(f"{len(df_clean)} rows, columns: {list(df_clean.columns)}")

    return df_clean

In [5]:
# === [3] Giữ những cột cần cho gợi ý ===
df = df.dropna(subset=["title", "content"])
df = df.reset_index(drop=True)

In [6]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.2 MB/s eta 0:00:00


In [7]:
import faiss
# Cần import thêm thư viện cho TF-IDF và FAISS nâng cấp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import os

# Install FAISS if not already installed
!pip install faiss-cpu

# --- Khởi tạo Model ---
model = SentenceTransformer("distiluse-base-multilingual-cased-v2")
D = model.get_sentence_embedding_dimension()

# --- Xây dựng Vectors ---
texts = df["title"] + " " + df["content"]

# 1. EMITTING EMBEDDINGS (Semantic Vectors)
print("1. Đang tính/tải Semantic Embeddings...")
# Compute embeddings and store them in df["embedding"]
article_embeddings = model.encode(texts.tolist(), normalize_embeddings=True, show_progress_bar=True).astype('float32')
df["embedding"] = article_embeddings.tolist()
print("   Semantic Embeddings đã được tính và lưu vào df['embedding'].")

# 2. XÂY DỰNG TF-IDF VECTORIZER
print("2. Đang xây dựng TF-IDF Vectorizer...")
# Sử dụng 1-gram và 2-gram cho tiếng Việt để bắt cụm từ tốt hơn
tfidf_vectorizer = TfidfVectorizer(
    lowercase=True,
    ngram_range=(1, 2),
    max_df=0.8, # Bỏ qua các từ quá phổ biến
    min_df=5 # Bỏ qua các từ quá hiếm
)
# Ma trận TF-IDF cho nội dung
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)
print(f"   Ma trận TF-IDF kích thước: {tfidf_matrix.shape}")


# 3. XÂY DỰNG FAISS INDEX (Sử dụng IndexIVFFlat để tăng tốc)
def build_advanced_faiss(embeddings: np.ndarray, faiss_path: str):
    print("3. Đang xây dựng IndexIVFFlat (FAISS Nâng cấp)...")
    N_CLUSTERS = 100

    # 3.1. Tạo IndexIVFFlat
    quantizer = faiss.IndexFlatIP(D) # Index dùng để phân cụm (quantizer)
    index = faiss.IndexIVFFlat(quantizer, D, N_CLUSTERS, faiss.METRIC_INNER_PRODUCT)

    # 3.2. Huấn luyện Index (Cần một phần nhỏ dữ liệu để phân cụm)
    # Lấy mẫu ngẫu nhiên 5000 vector để huấn luyện (nếu dữ liệu nhỏ hơn, dùng toàn bộ)
    sample_size = min(len(embeddings), 5000)
    faiss.normalize_L2(embeddings) # Normalization cần cho IndexFlatIP
    index.train(embeddings[:sample_size])

    # 3.3. Thêm dữ liệu vào Index
    index.add(embeddings)
    index.nprobe = 10 # Số lượng cụm cần quét (tăng lên để chính xác hơn, giảm để nhanh hơn)

    # 3.4. Lưu Index
    faiss.write_index(index, faiss_path)
    print(f"   FAISS Index IVFFlat (ntotal={index.ntotal}) đã được lưu.")
    return index

# Define FAISS_SNAPSHOT and build the index
FAISS_SNAPSHOT = "faiss_index.bin"
index = build_advanced_faiss(article_embeddings, FAISS_SNAPSHOT)

# Tách embedding khỏi df để tiết kiệm bộ nhớ
# df_metadata đã được tạo ở các bước trước (không có cột 'embedding')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

1. Đang tính/tải Semantic Embeddings...


Batches:   0%|          | 0/88 [00:00<?, ?it/s]

   Semantic Embeddings đã được tính và lưu vào df['embedding'].
2. Đang xây dựng TF-IDF Vectorizer...
   Ma trận TF-IDF kích thước: (2791, 57071)
3. Đang xây dựng IndexIVFFlat (FAISS Nâng cấp)...
   FAISS Index IVFFlat (ntotal=2791) đã được lưu.


In [8]:
from scipy.sparse import csr_matrix
def recommend_by_keyword_advanced(
    df_metadata: pd.DataFrame,
    model: SentenceTransformer,
    faiss_index: faiss.Index,
    tfidf_vectorizer: TfidfVectorizer,
    tfidf_matrix: csr_matrix, # Cần ma trận TF-IDF của tất cả bài báo
    query_keyword: str,
    top_n: int = 5,
    category_filter: str = None,
    top_k_faiss: int = 200 # Tăng nhẹ Top K để hỗ trợ TF-IDF
) -> pd.DataFrame:

    if not query_keyword:
        return pd.DataFrame()

    print(f"\n\u2009\u2505\u2009 Đang tìm kiếm cho: '{query_keyword}' (Lọc chuyên mục: {category_filter or 'None'})")

    # 1. TẠO QUERY VECTOR (Semantic & TF-IDF)
    query_sem_vec = model.encode(query_keyword, normalize_embeddings=True).astype('float32').reshape(1, -1)
    query_tfidf_vec = tfidf_vectorizer.transform([query_keyword]) # Vector TF-IDF cho từ khóa

    # 2. TÌM KIẾM FAISS (Lấy ứng viên tiềm năng)
    D, I = faiss_index.search(query_sem_vec, top_k_faiss)

    # Lấy metadata của Top K ứng viên
    indices = I[0].tolist()
    cands = df_metadata.iloc[indices].copy()
    cands["sim_sem"] = D[0].tolist() # Semantic Similarity

    # 3. TÍNH TF-IDF SIMILARITY (Chỉ tính trên tập Top K)
    # Lấy ma trận TF-IDF của các ứng viên
    tfidf_cands_matrix = tfidf_matrix[indices]

    # Tính cosine similarity giữa query TF-IDF và ma trận TF-IDF ứng viên
    sim_tfidf = cosine_similarity(query_tfidf_vec, tfidf_cands_matrix).ravel()
    cands["sim_tfidf"] = sim_tfidf

    # 4. LỌC CỨNG (Sửa lỗi Logic: Category)
    filtered_cands = cands.copy()
    if category_filter:
        category_filter_lower = category_filter.strip().lower()
        filtered_cands = filtered_cands[
            filtered_cands["category"].str.contains(category_filter_lower, na=False)
        ]

    if filtered_cands.empty:
        # (Giữ nguyên logic chẩn đoán đã sửa)
        top_cands_categories = cands["category"].value_counts().head(5).index.tolist()
        print(f"    (Diagnostic: 5 chuyên mục phổ biến nhất trong Top {top_k_faiss} ứng viên FAISS: {top_cands_categories})")
        print(f"⚠️ Không tìm thấy kết quả nào trong chuyên mục '{category_filter}' từ Top {top_k_faiss} ứng viên FAISS.")
        return pd.DataFrame()

    # 5. RERANKING & SCORE TỔNG HỢP
    # Xử lý thời gian (Recency)
    filtered_cands["published_time_parsed"] = pd.to_datetime(
        filtered_cands["published_time"], errors="coerce"
    )
    recency_norm = filtered_cands["published_time_parsed"].rank(pct=True, ascending=False).fillna(0.0)
    filtered_cands["recency_norm"] = recency_norm

    # CÔNG THỨC SCORE MỚI (Ưu tiên Semantic > TF-IDF > Recency)
    W_SEM, W_TFIDF, W_REC = 0.5, 0.35, 0.15
    filtered_cands["score"] = (
        W_SEM * filtered_cands["sim_sem"].astype(float).fillna(0) +
        W_TFIDF * filtered_cands["sim_tfidf"].astype(float).fillna(0) +
        W_REC * filtered_cands["recency_norm"]
    )

    # 6. CHỌN TOP N
    top_df = filtered_cands.sort_values(by="score", ascending=False).head(top_n)

    # 7. Hiển thị tất cả các điểm thành phần
    return top_df[[
        "title", "url", "category", "sim_sem", "sim_tfidf", "recency_norm", "score"
    ]]


In [20]:
# --- CHẠY THỬ NGHIỆM VỚI CODE MỚI ---
print("\n--- CHẠY THỬ NGHIỆM VỚI MÔ HÌNH CẢI TIẾN ---")

# Temporary fix: Rename 'section' column to 'category' if it exists
# This is a workaround because the 'df' DataFrame does not have a 'category' column
# but it has a 'section' column which is intended to be used as category.
if 'section' in df.columns and 'category' not in df.columns:
    df_for_recommendation = df.rename(columns={'section': 'category'})
elif 'section' not in df.columns and 'category' not in df.columns:
    print("Warning: Neither 'section' nor 'category' column found in DataFrame. Category filtering will not work.")
    df_for_recommendation = df.copy()
    df_for_recommendation['category'] = None # Add a dummy column to prevent further errors if category operations are attempted.
else:
    df_for_recommendation = df.copy()

# Gọi hàm với các tham số mới
result_ngap = recommend_by_keyword_advanced(
    df_metadata=df_for_recommendation,
    model=model,
    faiss_index=index,
    tfidf_vectorizer=tfidf_vectorizer,
    tfidf_matrix=tfidf_matrix,
    query_keyword="ngập",
    top_n=5,
    category_filter=None # Bỏ lọc chuyên mục để tìm kết quả "ngập"
)

if not result_ngap.empty:
    print("\n✨ Kết quả gợi ý cải tiến cho từ khóa: 'ngập' (Top 5)")
    print(result_ngap)



--- CHẠY THỬ NGHIỆM VỚI MÔ HÌNH CẢI TIẾN ---

 ┅  Đang tìm kiếm cho: 'ngập' (Lọc chuyên mục: None)

✨ Kết quả gợi ý cải tiến cho từ khóa: 'ngập' (Top 5)
                                                  title  \
282   Dự kiến bảng lương giáo viên của Bộ Giá...   
2228                                Đêm đầu tiên sau lũ   
1532   Sáng kiến Khoa học 2025 nhận hồ sơ đến ngày 30/3   
1555  'Đổi mới sáng tạo phải đóng góp 3% vào tăng tr...   
1556  'Tạo thông thoáng cho doanh nghiệp trong công ...   

                                                    url  category   sim_sem  \
282   https://vnexpress.net/du-kien-bang-luong-giao-...       NaN  0.144780   
2228  https://vnexpress.net/dem-dau-tien-sau-lu-4950...  doi-song  0.060672   
1532  https://vnexpress.net/sang-kien-khoa-hoc-2025-...  khoa-hoc  0.045170   
1555  https://vnexpress.net/doi-moi-sang-tao-phai-do...  khoa-hoc  0.036969   
1556  https://vnexpress.net/tao-thong-thoang-cho-doa...  khoa-hoc  0.029600   

      sim_tfidf 

In [21]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

def recommend_by_keyword(df, keyword, top_n=5):
    """
    Gợi ý bài giống với từ khóa hoặc tiêu đề nhập vào.
    Ưu tiên bài có thời gian đăng mới hơn.
    """
    if "embedding" not in df.columns:
        raise ValueError("⚠️ DataFrame chưa có cột 'embedding' (chưa chạy embedding).")

    print(f"\n🔍 Đang tìm bài liên quan tới: '{keyword}'")

    # Tạo embedding cho từ khóa nhập
    model = SentenceTransformer("distiluse-base-multilingual-cased-v2")
    query_vec = model.encode(keyword)

    # Make a copy to avoid modifying the original DataFrame with temporary columns
    # and potential SettingWithCopyWarning
    df_temp = df.copy()

    # Tính cosine similarity
    df_temp["similarity_kw"] = df_temp["embedding"].apply(lambda emb: cosine_similarity([query_vec], [emb])[0][0])

    # Nếu có thời gian đăng, chuyển sang datetime để sắp xếp
    if "published_time" in df_temp.columns:
        df_temp["published_time_parsed"] = pd.to_datetime(df_temp["published_time"], errors="coerce")
    else:
        df_temp["published_time_parsed"] = pd.NaT

    # Lọc top N theo similarity trước, sau đó sắp xếp theo ngày giảm dần
    top_df = df_temp.sort_values(by=["similarity_kw", "published_time_parsed"], ascending=[False, False]).head(top_n)

    print(f"\n✨ Kết quả gợi ý cho từ khóa: '{keyword}' (Top {top_n})\n")
    for i, row in top_df.iterrows():
        pub = row["published_time_parsed"].strftime("%d/%m/%Y") if pd.notna(row["published_time_parsed"]) else "N/A"
        print(f"- [{pub}] {row['title']}  — similarity: {row['similarity_kw']:.3f}")
        print(f"  {row['url']}\n")

    cols_to_return = ["url", "title", "published_time", "similarity_kw"]
    if "section" in top_df.columns:
        cols_to_return.insert(2, "section") # Insert 'section' at index 2
    else:
        top_df["section"] = None # Add a column with None if it doesn't exist

    return top_df[cols_to_return]

In [24]:
# === [7] In kết quả ===
print("\n===== KẾT QUẢ GỢI Ý ====")
# Use out_df from the previous recommendation call
if 'out_df' in locals() and not out_df.empty:
    for index, row in out_df.iterrows():
        url = row['url']
        title = row['title']
        sim = row['similarity_kw']
        time = row['published_time'] # Use original published_time string for output
        print(f"- {title} ({time}) — similarity: {sim:.3f}")
        print(f"  {url}")
else:
    print("Không có kết quả gợi ý nào được tạo ra.")

# === [8] Xuất JSON để web demo đọc ===
if 'out_df' in locals() and not out_df.empty:
    json_data = []
    for index, row in out_df.iterrows():
        json_data.append({
            "url": row['url'],
            "title": row['title'],
            "similarity": float(row['similarity_kw']),
            "published_time": row['published_time']
        })
    with open("recommendations_from_pseudo.csv.json", "w", encoding="utf8") as f:
        json.dump(json_data, f, ensure_ascii=False, indent=2)
    print("\nĐã lưu recommendations_from_pseudo.csv.json")
else:
    print("Không có dữ liệu để xuất JSON.")


===== KẾT QUẢ GỢI Ý ====
- Dự kiến bảng lương giáo viên của Bộ Giáo dục mới và chi tiết nhất (2025-09-19T00:00:00+07:00) — similarity: 0.145
  https://vnexpress.net/du-kien-bang-luong-giao-vien-cua-bo-giao-duc-moi-va-chi-tiet-nhat-4940807.html
- Sự tích thần giữ của (nan) — similarity: 0.131
  https://cotich.net/su-tich-than-giu-cua-a337.html
- Chàng Ngốc được kiện (nan) — similarity: 0.129
  https://cotich.net/chang-ngoc-duoc-kien-a336.html
- Đơn xin chôn Trâu (nan) — similarity: 0.128
  https://cotich.net/don-xin-chon-trau-a893.html
- Văn học dân gian (nan) — similarity: 0.126
  https://cotich.net/van-hoc-dan-gian.html
- Miệng kẻ sang (nan) — similarity: 0.113
  https://cotich.net/mieng-ke-sang-a901.html
- Ý nghĩa và bài học rút ra từ truyện cổ tích Cây tre trăm đốt (nan) — similarity: 0.113
  https://cotich.net/truyen-co-tich-cay-tre-tram-dot-a6.html
- Giàu vì bạn sang vì vợ: Giải thích ý nghĩa và bài học (nan) — similarity: 0.111
  https://cotich.net/giau-vi-ban-sang-v

In [23]:
# Define viewed_indices (example: indices of articles the user has viewed)
# These should be actual indices from your DataFrame 'df'
viewed_indices = [282, 2228, 1532] # Example indices from previously recommended articles

# Ensure article_embeddings and D (embedding dimension) are available from previous steps

if viewed_indices:
    # Get embeddings for the viewed articles
    viewed_embeddings = article_embeddings[viewed_indices]
    # Compute the user profile by averaging the embeddings
    user_profile = np.mean(viewed_embeddings, axis=0)
    print(f"User profile computed from {len(viewed_indices)} viewed articles. Shape: {user_profile.shape}")
else:
    user_profile = np.zeros(D) # D is the embedding dimension, previously defined
    print("No viewed articles, user profile initialized as zero vector.")

# You can now use 'user_profile' for personalized recommendations.

User profile computed from 3 viewed articles. Shape: (512,)


nhập thử từ khóa

In [25]:
keywords = ["ngập", "tàu biển", "hàng hải"]
for kw in keywords:
    recommend_by_keyword(df, kw, top_n=5)

out_df = recommend_by_keyword(df, "ngập", top_n=10)
out_df.to_json("recommend_ngap.json", orient="records", force_ascii=False, indent=2)

print("\n✅ Đã lưu ra file recommend_ngap.json")


🔍 Đang tìm bài liên quan tới: 'ngập'

✨ Kết quả gợi ý cho từ khóa: 'ngập' (Top 5)

- [19/09/2025] Dự kiến bảng lương giáo viên của Bộ Giáo dục mới và chi tiết nhất  — similarity: 0.145
  https://vnexpress.net/du-kien-bang-luong-giao-vien-cua-bo-giao-duc-moi-va-chi-tiet-nhat-4940807.html

- [N/A] Sự tích thần giữ của  — similarity: 0.131
  https://cotich.net/su-tich-than-giu-cua-a337.html

- [N/A] Chàng Ngốc được kiện  — similarity: 0.129
  https://cotich.net/chang-ngoc-duoc-kien-a336.html

- [N/A] Đơn xin chôn Trâu  — similarity: 0.128
  https://cotich.net/don-xin-chon-trau-a893.html

- [N/A] Văn học dân gian  — similarity: 0.126
  https://cotich.net/van-hoc-dan-gian.html


🔍 Đang tìm bài liên quan tới: 'tàu biển'

✨ Kết quả gợi ý cho từ khóa: 'tàu biển' (Top 5)

- [30/09/2025] Gia đình phản đối theo ngành Điều khiển tàu biển vì sợ vất vả  — similarity: 0.297
  https://vnexpress.net/gia-dinh-phan-doi-theo-nganh-dieu-khien-tau-bien-vi-so-vat-va-4942581.html

- [22/03/2025] 

In [26]:
# -------------------------------------------------------------
# [4] Gợi ý bài viết theo từ khóa nhập tay
# -------------------------------------------------------------
def recommend_by_keyword(df, model, top_n=5, category_filter=None, export_json=True):
    """
    Cho phép người dùng nhập từ khóa hoặc tiêu đề -> trả về bài tương tự.
    Hỗ trợ:
      - Lọc theo category (nếu nhập vào)
      - Sắp xếp theo độ tương đồng & ngày đăng mới nhất
      - Xuất ra JSON
    """
    print("\n==============================")
    print(" 🔎  GỢI Ý BÀI VIẾT THEO TỪ KHÓA ")
    print(" Gõ 'exit' để thoát.")
    print("==============================\n")

    while True:
        keyword = input("Nhập từ khóa hoặc tiêu đề bài viết: ").strip()
        if keyword.lower() == "exit":
            print("👋 Kết thúc tìm kiếm.")
            break
        if not keyword:
            print("⚠️ Vui lòng nhập ít nhất một từ khóa.\n")
            continue

        query_vec = model.encode(keyword)
        df["similarity"] = df["embedding"].apply(lambda emb: cosine_similarity([query_vec], [emb])[0][0])

        # Lọc category nếu được chọn
        filtered_df = df.copy()
        if category_filter:
            filtered_df = filtered_df[filtered_df["category"].str.contains(category_filter, case=False, na=False)]

        # Parse datetime nếu có
        filtered_df["published_time_parsed"] = pd.to_datetime(filtered_df["published_time"], errors="coerce")

        # Sắp xếp kết quả
        top_df = filtered_df.sort_values(
            by=["similarity", "published_time_parsed"], ascending=[False, False]
        ).head(top_n)

        print(f"\n✨ Kết quả gợi ý cho: '{keyword}' (Top {top_n})\n")
        for _, row in top_df.iterrows():
            d = row["published_time_parsed"].strftime("%d/%m/%Y") if pd.notnull(row["published_time_parsed"]) else "N/A"
            print(f"- [{d}] {row['title']} — {row['category'] or 'N/A'} — similarity: {row['similarity']:.3f}")
            print(f"  {row['url'] or 'No URL'}\n")

        if export_json:
            out_path = f"recommend_{keyword.replace(' ', '_')}.json"
            top_df.to_json(out_path, orient="records", force_ascii=False, indent=2)
            print(f"📁 Đã lưu vào: {out_path}\n")

        # Cho phép người dùng tiếp tục
        print("───────────────────────────────\n")

        try:
            top_n = int(input("Nhập số lượng bài muốn hiển thị (ví dụ 5, 10): ").strip())
            break
        except ValueError:
            print("⚠️ Vui lòng nhập số nguyên hợp lệ.\n")

    category_filter = input("Nhập tên chuyên mục nếu muốn lọc (nhấn Enter để bỏ qua): ").strip() or None

    recommend_by_keyword(df, model, top_n=top_n, category_filter=category_filter, export_json=True)

# Task
Define `viewed_indices` and compute `user_profile` by averaging the embeddings of the articles corresponding to `viewed_indices`, and insert this code before the cell `CyNwowUMspi6`.

## Define `viewed_indices` and `user_profile`

### Subtask:
Create a new code cell before the selected cell (`CyNwowUMspi6`) to define `viewed_indices` (e.g., with sample article indices or an empty list) and compute `user_profile` by averaging the embeddings of these `viewed_indices`.


## Summary:

### Insights or Next Steps
*   The next step involves the actual implementation of defining `viewed_indices` and computing `user_profile`.
*   The `user_profile`, once computed by averaging the embeddings of viewed articles, will be a critical input for subsequent personalized recommendation or content filtering tasks.
